# Machine Learning - Assignment 2

## Decision tree induction algorithm for classification tasks

The aim of the assignment is to:

* Implement a decision tree induction algorithm for classification tasks.
* Make sure it works for real valued features and nominal features (categorical features without rank, e.g., red - blue - green).
* Test the algorithm on 3 datasets.

Follow the instructions and implement what is missing to complete the assignment. Some functions have been started to help you a little bit with the inputs or outputs of the function.

**Note:** You might need to go back and forth during your implementation of the code. The structure is set up to make implementation easier, but how you return values from the different functions might vary, and you might find yourself going back and change something to make it easier later on.

## Assignment preparations

We help you out with importing the libraries.

**IMPORTANT NOTE:** You may not import any more libraries than the ones already imported!

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Decision tree model

The main objective is to implement the decision tree model. The implemented decision tree needs to be recursive model, that is, it should be implemented general enough to call itself in order to grow. "Growing" a tree refers to the same thing as "training" a model.

As said in the introduction, the structure is set up to help with implementation, but the nature of this model makes it a bit harder to implement function-by-function. You will most likely go back and forth between these first tasks.

### 1) Grow Tree

We will start with the main function of the decision tree, the "growing" function. 

This function should be called when creating the model, but also from inside itself. It is responible for creating all the nodes and leafs in the tree.

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None): # The star '*' can be removed
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

def grow_tree(X, y, number_of_features, max_depth, min_samples_for_split, depth=0):
    """
    X: This is the input data (features)
    y: This is the target labels (classes)
    number_of_features: is the number of features to consider when looking for the best split. This is useful
                        in for example ensamble models or random forest. Using less features reduce overfitting, and can help speed up training.
    max_depth: is the maximum depth of the tree
    min_samples_for_split: is the minimum number of samples required to split an internal node.
    depth: is the current depth of the tree, needed for the recursion.
    """
    nbr_samples = len(X)
    nbr_features = len(X[0])
    nbr_of_unique_labels = np.unique(y)

    if (depth >= max_depth or len(nbr_of_unique_labels) == 1 or nbr_samples <= min_samples_for_split or homogeneous(y)):
        leaf_value = most_common_label(y)
        return Node(value=leaf_value)
    
    number_of_features = X.shape[1] if not number_of_features else min(X.shape[1], number_of_features)
    feat_idx = np.random.choice(nbr_features, number_of_features, replace=False)
    best_feature, best_threshold = bestSplit(X, y, feat_idx)

    left_idxs = []
    right_idxs = []
    for i, x in enumerate(X[:, best_feature]):
        if x <= best_threshold:
            left_idxs.append(i)
        elif x > best_threshold:
            right_idxs.append(i)

    left_idxs = np.array(left_idxs)
    right_idxs = np.array(right_idxs)
    
    left = grow_tree(X[left_idxs, :], y[left_idxs], number_of_features, max_depth, min_samples_for_split, depth + 1)
    right = grow_tree(X[right_idxs, :], y[right_idxs], number_of_features, max_depth, min_samples_for_split, depth + 1)
    return Node(best_feature, best_threshold, left, right)

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None): # The star '*' can be removed
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

def grow_tree(X, y, number_of_features, max_depth, min_samples_for_split, depth=0):
    """
    X: This is the input data (features)
    y: This is the target labels (classes)
    number_of_features: is the number of features to consider when looking for the best split. This is useful
                        in for example ensamble models or random forest. Using less features reduce overfitting, and can help speed up training.
    max_depth: is the maximum depth of the tree
    min_samples_for_split: is the minimum number of samples required to split an internal node.
    depth: is the current depth of the tree, needed for the recursion.
    """
    nbr_samples = len(X)
    nbr_features = len(X[0])
    nbr_of_unique_labels = np.unique(y)

    if (depth >= max_depth or len(nbr_of_unique_labels) == 1 or nbr_samples <= min_samples_for_split or homogeneous(y)):
        leaf_value = most_common_label(y)
        return Node(value=leaf_value)
    
    number_of_features = X.shape[1] if not number_of_features else min(X.shape[1], number_of_features)
    feat_idx = np.random.choice(nbr_features, number_of_features, replace=False)
    best_feature, best_threshold = bestSplit(X, y, feat_idx)

    left_idxs = []
    right_idxs = []
    for i, x in enumerate(X[:, best_feature]):
        if x <= best_threshold:
            left_idxs.append(i)
        elif x > best_threshold:
            right_idxs.append(i)

    left_idxs = np.array(left_idxs)
    right_idxs = np.array(right_idxs)
    
    left = grow_tree(X[left_idxs, :], y[left_idxs], number_of_features, max_depth, min_samples_for_split, depth + 1)
    right = grow_tree(X[right_idxs, :], y[right_idxs], number_of_features, max_depth, min_samples_for_split, depth + 1)
    return Node(best_feature, best_threshold, left, right)




def homogeneous(y, threshold=0.9):
    most_common = most_common_label(y)
    count = np.sum(y == most_common)
    return count / len(y) >= threshold




def bestSplit(X, y, feat_idxs):
    best_gain = -1
    split_idx, split_threshold = None, None

    for feat_idx in feat_idxs:
        X_column = X[:, feat_idx]
        thresholds = np.unique(X_column)

        for thr in thresholds:
            gain = information_gain(y, X_column, thr)

            if gain > best_gain:
                best_gain = gain
                split_idx = feat_idx
                split_threshold = thr

    return split_idx, split_threshold

def information_gain(y, X_column, thr):
    parent_entropy = entropy(y)

    left_side = []
    right_side = []
    for i, x in enumerate(X_column):
        if x <= thr:
            left_side.append(i)
        elif x > thr:
            right_side.append(i)

    left_idx = np.array(left_side)
    right_idx = np.array(right_side)

    left_side_idxs_len, right_side_idxs_len = len(left_idx), len(right_idx)

    if left_side_idxs_len == 0 or right_side_idxs_len == 0:
        return 0
    
    n = len(y)
    left_side_entropy = entropy(y[left_idx])
    right_side_entropy = entropy(y[right_idx])
    
    child_entropy = ((left_side_idxs_len / n) * left_side_entropy) + ((right_side_idxs_len / n) * right_side_entropy)
    
    information_gain = parent_entropy - child_entropy
    return information_gain 

def gini(y):
    hist = np.bincount(y.astype(int))
    ps = hist / len(y)
    summ = 0
    for p in ps:
        if p > 0:
            summ += (2*p)*(1-p)
    return summ

def giniSqrt(y):
    hist = np.bincount(y.astype(int))
    ps = hist / len(y)
    summ = 0
    for p in ps:
        if p > 0:
            summ += (2*p)*(1-p)
    return (np.sqrt(summ))

def entropy(y):
    hist = np.bincount(y.astype(int))
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

def most_common_label(y):
    counter = {}
    for label in y:
        if label in counter:
            counter[label] += 1
        else:
            counter[label] = 1

    mx = float('-inf')
    ky = 0
    for key, val in counter.items():
        if val > mx:
            mx = val
            ky = key

    return ky


predictions: ['2' '2' '1' '1' '2' '2' '2' '2' '1' '1' '1' '1' '1' '1' '3' '3' '2' '1'
 '2' '2' '3' '2' '2' '2' '2' '2' '1' '1' '3' '1' '3' '1' '1' '3' '2' '3']
accuracy: 0.9444444444444444


### 2) Growth stopping conditions (or stopping criterias)

The "grow_tree" function needs some way of stop growing, otherwise it will grow indefinitely. We will adress this issue here.

The trees stopping criterias needs to handle the following:

1) When a node has only datapoints of a single class.

2) Prevent the tree from growing to large, i.e., a max depth.

3) Prevent the tree nodes from becoming to small.

4) Prevent the tree from growing when the node is large (has a lot of datapoints) but it is very unbalanced. This is an extention to case 1.

Can you think of some other stopping criterias that is good to have? 

In [37]:
# TODO: Change the name of the functions and implement them as you see fit.

def stop1():
    pass 

def stop2():
    pass 

#... # Add more stopping criterias if needed. Don't forget to use them when growing the tree!

### 3) Best feature for splitting nodes

When we are growing the tree, we need to decide how we are going to split a node into two new nodes. This is achived by looking at the features of the data in the node and calculate the best feature to split on.

Here you have a choice:

* Split using **Information Entropy**
* Split using **Gini Impurity**

Finish the function below using Information Entropy or Gini Impurity.

**Note:** Your code should be able to handle both real and categorical features!

In [38]:
# TODO: Change the name to which ever you chose.
# def func_name(...):

#     # TODO: Implement the rest of this function.
#     # NOTE: Do not forget to have support for real and categorical features.

#     return ...

### 4) Split data

When growing the tree, we need to split the data multiple times, and what we decide to split varies a lot. It is similar to splitting data into train and test sets (remember from assignment 1), but we split the data based on the best feature for growing a good tree.

**IMPORTANT NOTE:** To calculate binary splits for real-valued features, the following rule must be applied: an instance with a feature value lower than the mean feature value follows the left edge from the split node while all other instances follow the right edge from the split node.

In [39]:
# def split_data(data, ...):

#     # TODO: Implement the rest of this function.

#     return ...


### 5) Predict with tree model

Finally, when we have grown our tree, we would like to use it for prediction. When using the tree for prediction, we traverse the tree for each datapoint untill we land in a leaf node.

In [48]:
def traverse_tree(x, node):
    if node.value is not None:
        return node.value
    
    if x[node.feature] <= node.threshold:
        return traverse_tree(x, node.left)
    return traverse_tree(x, node.right)

## Test decision tree model, compare with scikit learn, and plot dataset results

In the last part of the lab, you are going to test your tree code and compare it to scikit learn. The goal is not to be better than an established library, but to give you an indication about if you are on the right track.

You will need to plot the results from your model and the scikit learn model using matplotlib. We suggest a simple but informative bar-charts.

To make the comparison fair, you should train and test both your decision tree algorithm and the scikit learn at least 5 times, and shuffle the data each time before splitting the data into a train and test set.

The datasets are:

* Wine - (https://archive.ics.uci.edu/dataset/109/wine)
* Heart disease - (https://archive.ics.uci.edu/dataset/45/heart+disease)
* Car - (https://archive.ics.uci.edu/dataset/19/car+evaluation)

**IMPORTANT NOTE 1:** Take note of the feature types in the datasets, some features are numerical in value but are in fact categorical features. Be sure to handle these features correctly!

**IMPORTANT NOTE 2:** In this assignment it helps to add an additional header with information about the features and if they are nominal (n) or real (r) features.

In [41]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

You may use the "**accuracy_score**" function from scikit learn (imported above) to compare the performance of your own and scikit learns models.

See below for an example use.

In [42]:
y_true = [1,1,1,1,1] # Pretend labels
y_pred = [1,1,2,2,1] # Pretend prediction

accuracy_score(y_true, y_pred)

0.6

### 6) Dataset 1: Wine

In [43]:
data_wine = pd.read_csv("wine.csv").to_numpy()
data_wine = data_wine[1:]


# TODO: Set up the data and split it into train and test-sets.
X = data_wine[:, :-1]
y = data_wine[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# TODO: Train and test your implemented tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model
number_of_features = X.shape[1]
root = grow_tree(X_train, y_train, number_of_features, max_depth=10, min_samples_for_split=2)
my_predictions = np.array([traverse_tree(x, root) for x in X_test])
print(f"my_predictions: {my_predictions}")


# TODO: Train and test scikit learns tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model
scikit_clf = DecisionTreeClassifier(criterion='entropy')
scikit_clf.fit(X_train, y_train)
scikit_predictions = scikit_clf.predict(X_test)
print(f"scikit_predictions: {scikit_predictions}")


# TODO: Do the above at least 5 times
# NOTE: Use loops here!

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(y_test, my_predictions)
print(f"predictions: {my_predictions}")
print(f"accuracy: {acc}")

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough


### 7) Dataset 2: Heart Disease

In [44]:
data_heart = pd.read_csv("heart.csv").to_numpy()

# TODO: Set up the data and split it into train and test-sets.

# TODO: Train and test your implemented tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Train and test scikit learns tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

### 8) Dataset 3: Car

In [45]:
data_car = pd.read_csv("car.csv").to_numpy()

# TODO: Set up the data and split it into train and test-sets.

# TODO: Train and test your implemented tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Train and test scikit learns tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

### 9) Training with normalized data on the wine-dataset

So far, we have trained our decision trees with "raw" data, i.e., we haven't done much preprocessing on the data.

Here we will do minor preprocessing on the data with the help of the scikit-learn library: https://scikit-learn.org/stable/modules/preprocessing.html

In [46]:
from sklearn import preprocessing

# TODO: Use the wine dataset from above and scale its features and labels between 0 and 1

# TODO: Run the code from the dataset and compare the preprocessed vs non-preprocessed data.
# NOTE: You can copy most of the workflow from the dataset code above to save you some time.
# NOTE: Use the same train/test split for your tree model for both the preprocessed vs non-preprocessed data.

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

# Questions for examination:

In addition to completing the assignment with all its tasks, you should also prepare to answer the following questions:

1) Why is growing the tree indefinitely such a bad idea? The performance would increase would it not?

2) Beside preventing the tree from growing to large, what is the purpose of 'stopping criterias'?

3) What is the difference between **Information Entropy** and **Gini Impurity**?

4) What are some pros about using decision trees?

5) Did preprocessing the data help with performance when using decision trees?

# Finished!

Was part of the setup incorrect? Did you spot any inconsistencies in the assignment? Could something improve?

If so, please write them and send via email and send it to:

* marcus.gullstrand@ju.se

Thank you!

In [47]:
# Performance testing
import time

def split(X_column, thr):
    # Timing the for loop
    start_time = time.time()
    la = []
    for i, x in enumerate(X_column):
        if x <= thr:
            la.append(i)
    left_id = np.array(la)
    for_loop_time = time.time() - start_time

    start_time = time.time()
    s = set()
    for i, x in enumerate(X_column):
        if x <= thr:
            s.add(i)
    ss = np.array(s)
    set_time = time.time() - start_time

    # Timing the argwhere method
    start_time = time.time()
    left_idxs = np.argwhere(X_column <= thr).flatten()
    argwhere_time = time.time() - start_time

    if not np.array_equal(left_id, left_idxs):
        print("ERROR")

    right_idxs = np.argwhere(X_column > thr).flatten()

    if for_loop_time < argwhere_time and for_loop_time < set_time:
        print("For loop is faster")
    elif argwhere_time < for_loop_time and argwhere_time < set_time:
        print("Argwhere is faster")
    else:
        print("Set is faster")